In [1]:
from sagar.crystal.derive import ConfigurationGenerator
from sagar.io.vasp import read_vasp
from sagar.crystal.structure import symbol2number as s2n
import numpy as np
from random import randint
import os
import shutil

In [2]:
def _get_sites(atoms, mag_atom, sub_atom):
    return [tuple(set([i, sub_atom]))  if i == mag_atom else (i,) for i in atoms ]

In [3]:
def _get_correspond(A,B):
    idx = []
    for ii in A:
        tmp = np.linalg.norm(ii - B,axis=1)
        q0,q1 = np.where(abs(tmp)<1e-3)[0],np.where(abs(tmp-1)<1e-3)[0]
        n0 = np.size(q0)
        if n0: 
            idx.append(q0[0]) 
        else: 
            idx.append(q1[0])
    return idx

In [4]:
symprec = 1e-3
comprec = 1e-3
mag_element = 'Cr'
mag_num = s2n(mag_element)
substitutes = np.setdiff1d(np.arange(1,89),mag_num)[randint(1,88)]
folder = 'all_mag_confi'

In [5]:
cell = read_vasp('POSCAR_not_work.vasp')
cg = ConfigurationGenerator(cell, symprec)
sites = _get_sites(list(cell.atoms), mag_num,substitutes)
mag_number = len(np.where(cell.atoms == mag_num)[0])
pos = cell.positions

In [6]:
if not os.path.exists('./'+folder):
    os.mkdir('./'+folder)
else:
    shutil.rmtree('./'+folder)
    os.mkdir('./'+folder)

In [7]:
idx = 0   
for sub_num in range(1, mag_number//2+1):
    confs = cg.cons_specific_cell(sites,e_num=(mag_number-sub_num,sub_num),symprec=symprec)
    for c, _ in confs:
        filename = os.path.join('./'+folder, 'INCAR' + str(idx))
        corre_idx = _get_correspond(pos, c.positions)
        _atoms = c.atoms[corre_idx]
        mag_para = ['-1' if i == mag_num else '1' for i in _atoms]
        mag_para = ' '.join(mag_para)
        with open(filename,'w') as f:
            f.writelines('MAG= ' + mag_para)
        idx += 1